<a href="https://colab.research.google.com/github/victor-roris/ML-learning/blob/master/QuestionAnswering/TAPAS-Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAPAS - Transformers

Link: https://huggingface.co/transformers/model_doc/tapas.html

The TAPAS model was proposed in [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://www.aclweb.org/anthology/2020.acl-main.398)  for answering questions about tabular data. Compared to BERT, TAPAS uses relative position embeddings and has 7 token types that encode tabular structure. TAPAS is pre-trained on the masked language modeling (MLM) objective on a large dataset comprising millions of tables from English Wikipedia and corresponding texts. For question answering, TAPAS has 2 heads on top: a cell selection head and an aggregation head, for (optionally) performing aggregations (such as counting or summing) among selected cells.

In [ ]:
!pip install transformers
!pip install torch-scatter

## Usage: fine-tuning

### STEP 1: Choose one of the 3 ways in which you can use TAPAS - or experiment
 
| Task | Example dataset | Description |
|:-----|:----------------|-------------|
| Conversational | SQA | Conversational, only cell selection questions |
| Weak supervision for aggregation | WTQ | Questions might involve aggregation, and the model must learn this given only the answer as supervision |
| Strong supervision for aggregation | WikiSQL-supervised | Questions might involve aggregation, and the model must learn this given the gold aggregation operator |



Initializing a model with a pre-trained base and randomly initialized classification heads from the model hub can be done as follows :

In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering

# for example, the base sized model with default SQA configuration
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base')

# or, the base sized model with WTQ configuration
config = TapasConfig.from_pretrained('google/tapas-base-finetuned-wtq')
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)

## or, the base sized model with WikiSQL configuration
# config = TapasConfig('google-base-finetuned-wikisql-supervised')
# model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['column_output_bias', 'output_weights', 'output_bias', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.weight', 'column_output_bias', 'output_weights', 'aggregation_classifier.bias', 'output_bias', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You can also experiment by defining any hyperparameters you want when initializing TapasConfig, and then create a TapasForQuestionAnswering based on that configuration.

In [ ]:
from transformers import TapasConfig, TapasForQuestionAnswering

# you can initialize the classification heads any way you want (see docs of TapasConfig)
config = TapasConfig(num_aggregation_labels=3, average_logits_per_cell=True, select_one_column=False)
# initializing the pre-trained base sized model with our custom classification heads
model = TapasForQuestionAnswering.from_pretrained('google/tapas-base', config=config)

Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.weight', 'column_output_bias', 'output_weights', 'aggregation_classifier.bias', 'output_bias', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Or you can use some pretrained model: https://huggingface.co/models?search=tapas

### STEP 2: Prepare your data in the SQA format

Second, no matter what you picked above, you should prepare your dataset in the SQA format. This format is a TSV/CSV file with the following columns:

- `id`: optional, id of the table-question pair, for bookkeeping purposes.
- `annotator`: optional, id of the person who annotated the table-question pair, for bookkeeping purposes.
- `position`: integer indicating if the question is the first, second, third,… related to the table. Only required in case of conversational setup (SQA). You don’t need this column in case you’re going for WTQ/WikiSQL-supervised.
- `question`: string
- `table_file`: string, name of a csv file containing the tabular data
- `answer_coordinates`: list of one or more tuples (each tuple being a cell coordinate, i.e. row, column pair that is part of the answer)
- `answer_text`: list of one or more strings (each string being a cell value that is part of the answer)
- `aggregation_label`: index of the aggregation operator. Only required in case of strong supervision for aggregation (the WikiSQL-supervised case)
- `float_answer`: the float answer to the question, if there is one (np.nan if there isn’t). Only required in case of weak supervision for aggregation (such as WTQ and WikiSQL)

The tables themselves should be present in a folder, each table being a separate csv file. Note that the authors of the TAPAS algorithm used conversion scripts with some automated logic to convert the other datasets (WTQ, WikiSQL) into the SQA format. The author explains this [here](https://github.com/google-research/tapas/issues/50#issuecomment-705465960). Interestingly, these conversion scripts are not perfect (the answer_coordinates and float_answer fields are populated based on the answer_text), meaning that WTQ and WikiSQL results could actually be improved.

### STEP 3: Convert your data into PyTorch tensors using TapasTokenizer

Third, given that you’ve prepared your data in this TSV/CSV format (and corresponding CSV files containing the tabular data), you can then use TapasTokenizer to convert table-question pairs into input_ids, attention_mask, token_type_ids and so on. Again, based on which of the three cases you picked above, TapasForQuestionAnswering requires different inputs to be fine-tuned:

| Task | Required inputs |
|:----:|:---------------:|
| Conversational |input_ids, attention_mask, token_type_ids, labels |
| Weak supervision for aggregation | input_ids, attention_mask, token_type_ids, labels, numeric_values, numeric_values_scale, float_answer |
| Strong supervision for aggregation | input ids, attention mask, token type ids, labels, aggregation_labels |



TapasTokenizer creates the labels, numeric_values and numeric_values_scale based on the answer_coordinates and answer_text columns of the TSV file. 

In [ ]:
from transformers import TapasTokenizer
import pandas as pd

model_name = 'google/tapas-base'
tokenizer = TapasTokenizer.from_pretrained(model_name)

data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 'Number of movies': ["87", "53", "69"]}
queries = ["What is the name of the first actor?", "How many movies has George Clooney played in?", "What is the total number of movies?"]
answer_coordinates = [[(0, 0)], [(2, 1)], [(0, 1), (1, 1), (2, 1)]]
answer_text = [["Brad Pitt"], ["69"], ["209"]]
table = pd.DataFrame.from_dict(data)

print("Table as dataframe: ")
display(table.head())
print()

inputs = tokenizer(table=table, queries=queries, 
                   answer_coordinates=answer_coordinates,
                   answer_text=answer_text, padding='max_length',
                   return_tensors='pt')

print("TapasTokenization output:")
for kei in inputs.keys():
  print(f" - {kei} :")
  print(f"  {inputs[kei]}")
  print()

Table as dataframe: 


,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69



TapasTokenization output:
 - input_ids :
  tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]])

 - labels :
  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

 - numeric_values :
  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])

 - numeric_values_scale :
  tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

 - token_type_ids :
  tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
 

Note that TapasTokenizer expects the data of the table to be text-only. You can use .astype(str) on a dataframe to turn it into text-only data. Of course, this only shows how to encode a single training example. It is advised to create a PyTorch dataset and a corresponding dataloader:

```python
import torch
import pandas as pd

tsv_path = "your_path_to_the_tsv_file"
table_csv_path = "your_path_to_a_directory_containing_all_csv_files"

class TableDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = data.iloc[idx]
        table = pd.read_csv(table_csv_path + item.table_file).astype(str) # be sure to make your table data text only
        encoding = self.tokenizer(table=table,
                                  queries=item.question,
                                  answer_coordinates=item.answer_coordinates,
                                  answer_text=item.answer_text,
                                  truncation=True,
                                  padding="max_length",
                                  return_tensors="pt"
        )
        # remove the batch dimension which the tokenizer adds by default
        encoding = {key: val.squeeze(0) for key, val in encoding.items()}
        # add the float_answer which is also required (weak supervision for aggregation case)
        encoding["float_answer"] = torch.tensor(item.float_answer)
        return encoding

    def __len__(self):
       return len(self.data)

data = pd.read_csv(tsv_path, sep='\t')
train_dataset = TableDataset(data, tokenizer)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32)

```

Note that here, we encode each table-question pair independently. This is fine as long as your dataset is not conversational. In case your dataset involves conversational questions (such as in SQA), then you should first group together the queries, answer_coordinates and answer_text per table (in the order of their position index) and batch encode each table with its questions. This will make sure that the prev_labels token types (see docs of TapasTokenizer) are set correctly. See this [notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/TAPAS/Fine_tuning_TapasForQuestionAnswering_on_SQA.ipynb) for more info.

### STEP 4: Train (fine-tune) TapasForQuestionAnswering

```python
from transformers import TapasConfig, TapasForQuestionAnswering, AdamW

# this is the default WTQ configuration
config = TapasConfig(
           num_aggregation_labels = 4,
           use_answer_as_supervision = True,
           answer_loss_cutoff = 0.664694,
           cell_selection_preference = 0.207951,
           huber_loss_delta = 0.121194,
           init_cell_selection_weights_to_zero = True,
           select_one_column = True,
           allow_empty_column_selection = False,
           temperature = 0.0352513,
)
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(2):  # loop over the dataset multiple times
   for idx, batch in enumerate(train_dataloader):
        # get the inputs;
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        float_answer = batch["float_answer"]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                       labels=labels, numeric_values=numeric_values, numeric_values_scale=numeric_values_scale,
                       float_answer=float_answer)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
```


## Usage: Interface

You can use a local trained model or some pretrained model: https://huggingface.co/models?search=tapas

In [3]:
 from transformers import TapasTokenizer, TapasForQuestionAnswering
 import pandas as pd

 model_name = 'google/tapas-base-finetuned-wtq'
 model = TapasForQuestionAnswering.from_pretrained(model_name)
 tokenizer = TapasTokenizer.from_pretrained(model_name)

Define the input table and queries

In [6]:
data = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"],
         'Number of movies': ["87", "53", "69"]}
queries = ["What is the name of the first actor?",
          "How many movies has George Clooney played in?",
          "What is the total number of movies?"]

table = pd.DataFrame.from_dict(data)
table

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69


Tokenize the input data

In [9]:
inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")

for kei in inputs.keys():
  print(f" - {kei} :")
  print(f"  {inputs[kei]}")
  print()

 - input_ids :
  tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2129, 2116,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]])

 - token_type_ids :
  tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]])

 - attention_mask :
  tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

Obtain the predictions

In [10]:
outputs = model(**inputs)

predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs,
        outputs.logits.detach(),
        outputs.logits_aggregation.detach()
)

Print out the results

In [11]:
# let's print out the results:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT"} # We use WTQ model > We have aggregation operations
aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
      cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
  print(query)
  if predicted_agg == "NONE":
    print("Predicted answer: " + answer)
  else:
    print("Predicted answer: " + predicted_agg + " > " + answer)

,Actors,Number of movies
0,Brad Pitt,87
1,Leonardo Di Caprio,53
2,George Clooney,69



What is the name of the first actor?
Predicted answer: Brad Pitt
How many movies has George Clooney played in?
Predicted answer: COUNT > 69
What is the total number of movies?
Predicted answer: SUM > 87, 53, 69
